In [1]:
import pandas as pd
import numpy as np

# DATA CLEANING

### <b>Ideas:<b>
    - Only look at people who were successfully iterviewed all 3 waves
    - Merge Wave1, Wave2 and Wave3 in order to look at the same population
    -
    -

    
    
    

In [32]:
Wave1 = pd.read_stata("Wave1.dta")
# display(Wave1.head(5))
# Wave1.info()

num_rows_start = Wave1.shape[0]

Wave1 = Wave1[Wave1['w1_nc_outcome'] == "Successfully Interviewed"]
deleted_rows = num_rows_start - Wave1.shape[0]

# display(Wave1.head(5))
# Wave1.info()
print("Number of people who were not interviewed in Wave 1: ", deleted_rows)

Number of people who were not interviewed in Wave 1:  0


In [33]:
Wave2 = pd.read_stata("Wave2.dta")
# display(Wave2.head(5))
# Wave2.info()

num_rows_start = Wave2.shape[0]

Wave2 = Wave2[Wave2['w2_nc_outcome'] == "Successfully Interviewed"]
deleted_rows = num_rows_start - Wave2.shape[0]

# display(Wave2.head(5))
# Wave2.info()
print("Number of people who were not interviewed in Wave 2: ", deleted_rows)

Number of people who were not interviewed in Wave 2:  1375


In [42]:
Wave3 = pd.read_stata("Wave3.dta")
# display(Wave3.head(5))
# Wave3.info()

num_rows_start = Wave3.shape[0]

Wave3 = Wave3[Wave3['w3_nc_outcome'] == "Successfully Interviewed"]
deleted_rows = num_rows_start - Wave3.shape[0]

# display(Wave3.head(5))
# Wave3.info()
print("Number of people who were not interviewed in Wave 3: ", deleted_rows)

Number of people who were not interviewed in Wave 3:  1968


We have now narrrowed down our dataset to only the people who were successfully interviewed.

In [37]:
frames = [Wave1, Wave2, Wave3]
temp = pd.merge(Wave1, Wave2, how='inner', on='pid')
df = pd.merge(temp, Wave3, how='inner', on='pid')

display(df.head(5))
print("Number of people who were successfully interviewed in all 3 waves: ", df.shape[0])

,pid,w1_nc_outcome,w1_nc_intrv_c,w1_nc_intrv_d,w1_nc_intrv_m,w1_nc_intrv_y,w1_nc_duration,w1_nc_dob_m,w1_nc_dob_y,w1_nc_gen,...,w3_nc_smkprice_ld,w3_nc_smkbrand_ld,w3_nc_smkbrand_ld_o,w3_nc_smkcigday,w3_nc_smkbrand_after,w3_nc_smkbrand_after_o,w3_nc_intres,w3_nc_intlng1,w3_nc_intlng2,w3_nc_intlng3
0,301067,Successfully Interviewed,609,15th,May,2020,15.2,July,1971.0,Woman,...,NaN,NaN,,NaN,NaN,,Friendly,Sepedi,NaN,NaN
1,301079,Successfully Interviewed,609,22nd,May,2020,15.37,February,1970.0,Woman,...,NaN,NaN,,NaN,NaN,,Friendly,Sepedi,Sesotho,NaN
2,301082,Successfully Interviewed,624,4th,June,2020,25.27,June,1989.0,Man,...,NaN,NaN,,NaN,NaN,,Friendly,IsiXhosa,NaN,NaN
3,301086,Successfully Interviewed,644,4th,June,2020,17.18,June,1972.0,Woman,...,NaN,NaN,,NaN,NaN,,Friendly,IsiZulu,NaN,NaN
4,301088,Successfully Interviewed,630,27th,May,2020,21.13,May,1978.0,Man,...,NaN,NaN,,NaN,NaN,,Hostile,IsiXhosa,English,NaN


Number of people who were successfully interviewed in all 3 waves:  4508


In [46]:
personal_info_cols = [  'pid','w1_nc_dob_y', 'w2_nc_dob_y', 'w3_nc_dob_y',
                        'w1_nc_dob_m', 'w2_nc_dob_m', 'w3_nc_dob_m', 
                        'w1_nc_gen', 'w2_nc_gen', 'w3_nc_gen',
                        'w1_nc_popgrp',  'w3_nc_popgrp']

df_personal = df[personal_info_cols].copy()
df_personal

,pid,w1_nc_dob_y,w2_nc_dob_y,w3_nc_dob_y,w1_nc_dob_m,w2_nc_dob_m,w3_nc_dob_m,w1_nc_gen,w2_nc_gen,w3_nc_gen,w1_nc_popgrp,w3_nc_popgrp
0,301067,1971.0,1971.0,1971.0,July,July,July,Woman,Woman,Woman,African/Black,African/Black
1,301079,1970.0,1970.0,1970.0,February,February,February,Woman,Woman,Woman,African/Black,African/Black
2,301082,1989.0,1989.0,1989.0,June,June,June,Man,Man,Man,African/Black,African/Black
3,301086,1972.0,1972.0,1972.0,June,June,June,Woman,Woman,Woman,African/Black,African/Black
4,301088,1978.0,1978.0,1978.0,May,May,May,Man,Man,Man,African/Black,African/Black
...,...,...,...,...,...,...,...,...,...,...,...,...
4503,799572,1989.0,1990.0,1989.0,August,August,August,Woman,Woman,Woman,African/Black,African/Black
4504,799619,1978.0,1978.0,1978.0,September,October,September,Man,Man,Man,African/Black,African/Black
4505,799644,1976.0,1976.0,1976.0,January,January,January,Woman,Woman,Woman,African/Black,African/Black
4506,799780,1996.0,1996.0,1996.0,July,July,July,Woman,Woman,Woman,Coloured,Coloured


There are several cases where the values are inconsistent across the different waves. I believe this is an input error since many of the entires will match. For example, the year of birth in record #4503 is (1989, 1990, 1989) yet the month of birth and gender is consistent (August, Woman). 
To clean this up, we should look at the triplet of columns (that we are interested in - birth date, gender, etc) and set all 3 values to whichever one appears most often.

In [47]:
for index, row in df_personal.iterrows():
    years = row[['w1_nc_dob_y', 'w2_nc_dob_y', 'w3_nc_dob_y']]
    months = row[['w1_nc_dob_m', 'w2_nc_dob_m', 'w3_nc_dob_m']]
    genders = row[['w1_nc_gen', 'w2_nc_gen', 'w3_nc_gen']]
    races = row[['w1_nc_popgrp',  'w3_nc_popgrp']]


    YearOfBirth = years.value_counts().idxmax()
    MonthOfBirth = months.value_counts().idxmax()
    Gender = genders.value_counts().idxmax()   
    Race = races.value_counts().idxmax()

    

    df_personal.loc[index, 'YearOfBirth'] = YearOfBirth
    df_personal.loc[index, 'MonthOfBirth'] = MonthOfBirth
    df_personal.loc[index, 'Gender'] = Gender
    df_personal.loc[index, 'Race'] = Race

In [48]:
df_personal = df_personal.drop(personal_info_cols, axis=1)
df_personal

,YearOfBirth,MonthOfBirth,Gender,Race
0,1971.0,July,Woman,African/Black
1,1970.0,February,Woman,African/Black
2,1989.0,June,Man,African/Black
3,1972.0,June,Woman,African/Black
4,1978.0,May,Man,African/Black
...,...,...,...,...
4503,1989.0,August,Woman,African/Black
4504,1978.0,September,Man,African/Black
4505,1976.0,January,Woman,African/Black
4506,1996.0,July,Woman,Coloured


In [49]:
df_dropped = df.drop(personal_info_cols, axis=1)
df_main = pd.concat([df_personal, df_dropped], axis=1)
df_main

,YearOfBirth,MonthOfBirth,Gender,Race,w1_nc_outcome,w1_nc_intrv_c,w1_nc_intrv_d,w1_nc_intrv_m,w1_nc_intrv_y,w1_nc_duration,...,w3_nc_smkprice_ld,w3_nc_smkbrand_ld,w3_nc_smkbrand_ld_o,w3_nc_smkcigday,w3_nc_smkbrand_after,w3_nc_smkbrand_after_o,w3_nc_intres,w3_nc_intlng1,w3_nc_intlng2,w3_nc_intlng3
0,1971.0,July,Woman,African/Black,Successfully Interviewed,609,15th,May,2020,15.2,...,NaN,NaN,,NaN,NaN,,Friendly,Sepedi,NaN,NaN
1,1970.0,February,Woman,African/Black,Successfully Interviewed,609,22nd,May,2020,15.37,...,NaN,NaN,,NaN,NaN,,Friendly,Sepedi,Sesotho,NaN
2,1989.0,June,Man,African/Black,Successfully Interviewed,624,4th,June,2020,25.27,...,NaN,NaN,,NaN,NaN,,Friendly,IsiXhosa,NaN,NaN
3,1972.0,June,Woman,African/Black,Successfully Interviewed,644,4th,June,2020,17.18,...,NaN,NaN,,NaN,NaN,,Friendly,IsiZulu,NaN,NaN
4,1978.0,May,Man,African/Black,Successfully Interviewed,630,27th,May,2020,21.13,...,NaN,NaN,,NaN,NaN,,Hostile,IsiXhosa,English,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4503,1989.0,August,Woman,African/Black,Successfully Interviewed,609,15th,May,2020,16.92,...,NaN,NaN,,NaN,NaN,,Friendly,Sepedi,NaN,NaN
4504,1978.0,September,Man,African/Black,Successfully Interviewed,616,21st,May,2020,18.9,...,5.0,Rudland & George (RG),,10.0,Pall Mall,,Friendly,Sepedi,Sesotho,NaN
4505,1976.0,January,Woman,African/Black,Successfully Interviewed,633,16th,June,2020,23.37,...,NaN,NaN,,NaN,NaN,,Friendly,IsiZulu,NaN,NaN
4506,1996.0,July,Woman,Coloured,Successfully Interviewed,614,10th,June,2020,16.5,...,80.0,Rothmans,,8.0,Rothmans,,Neither hostile nor friendly,Afrikaans,NaN,NaN


Above we can see the successful deletion of the duplicated DOB, gender and race columns, and the addition of the single columns with the correct values.